In [1]:
import pandas as pd
import errno    
import os
import numpy as np
import math
import time

'''
Author: Enrico Ceccolini
    Create a dataframe to be used as the base of a power prediction model
'''

datadir = "/datasets/eurora_data/db_local/"
#datadir = "/datasets/eurora_data/db/" #hdd

infile_jobs_to_nodes = datadir + "job_nodes.csv"

suffix = "_5sec_"
### select an interval from
## 1 settings wholeData
#interval_comment_whole = "WholeData"

### select an interval from
## 2 settings Andrea
interval_comment = "Andrea"
start_time = pd.to_datetime('2014-03-31')
end_time = pd.to_datetime('2014-05-01')
#infile_jobs = datadir + "april_long_jobs.csv"

## 3 settings Alina
interval_comment = "Alina"
start_time = pd.to_datetime('2014-03-31')
end_time = pd.to_datetime('2014-11-01')
train_start_time = pd.to_datetime('2014-03-31')
train_end_time = pd.to_datetime('2014-10-01')
test_end_time = pd.to_datetime('2014-11-01')




#infile_jobs = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_long_jobs_real_pow"
infile_jobs_trainset = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_trainset"
infile_jobs_testset = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_testset"

#infile_jobs_trainset = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_trainset_clear"
#infile_jobs_testset = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_testset_clear"

nodes=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64']

In [2]:
jobs_train = pd.read_csv(infile_jobs_trainset + ".csv", index_col=0)
print("train set contains {} records".format(jobs_train.shape[0]))
jobs_test = pd.read_csv(infile_jobs_testset + ".csv", index_col=0)
print("test set contains {} records".format(jobs_test.shape[0]))

train set contains 146901 records
test set contains 18224 records


In [3]:
jobs_to_nodes_whole_data = pd.read_csv(infile_jobs_to_nodes, index_col=0)
print("jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))
### clean the data
# remove jobs runned on the inexistent node 129
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data[jobs_to_nodes_whole_data['node_id'] != 129] 
# remove jobs with the same id that runned on the same node
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data.drop_duplicates(subset=['job_id_string', 'node_id'])
print("after the clean, jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

jobs_to_nodes_whole_data contains 469095 records
after the clean, jobs_to_nodes_whole_data contains 444610 records


In [43]:
###### NB eseguire solo se non si vogliono job sotto 5 sec

#jobs_train = jobs_train[pd.to_datetime(jobs_train['end_time']) - pd.to_datetime(jobs_train['run_start_time']) > np.timedelta64(5, 's')]
#print(jobs_train.shape[0])
#jobs_test = jobs_test[pd.to_datetime(jobs_test['end_time']) - pd.to_datetime(jobs_test['run_start_time']) > np.timedelta64(5, 's')]
#print(jobs_test.shape[0])

In [44]:
# train_whole: 167324 ; test_whole:29935 
# train_long: 39363 ; test_long: 11622

In [7]:
### merge the info from the two tables
merged_jobs_to_nodes = pd.merge(jobs_train, jobs_to_nodes_whole_data, how='left', on='job_id_string')
print("merged_jobs_to_nodes contains {} records".format(merged_jobs_to_nodes.shape[0]))

merged_jobs_to_nodes contains 161420 records


In [46]:
jobs_to_nodes_whole_data.head(1)

,node_id,job_id_string,ncpus,ngpus,nmics,mem_requested
job2node_id,,,,,,
1,17,498458.node129,16,0,0,14680064


In [47]:
jobs_train.head(1)

,job_id_string,job_name,queue,start_time,run_start_time,end_time,user,node_req,cpu_req,mem_req,...,real_pow_quality,n_2_1,n_2_2,n_3_1,job_tot_timepoints,job_timepoints,good_nodes,ncores_system,njobs_system,load_system
job_id,,,,,,,,,,,,,,,,,,,,,
205795,1006765.node129,442_1.pbs,parallel,2014-08-13 08:40:57,2014-08-13 10:19:44,2014-08-13 10:19:45,cdongmof,1,4,1048576,...,1.0,0,0,1,1,0,1,417,76,0


### good code

We want to obtain 5 dataframes:
- node
- user
- user_node
- user_job
- user_job_node

In [48]:
merged_jobs_to_nodes['core_avg'] = merged_jobs_to_nodes['real_pow'] / merged_jobs_to_nodes['cpu_req']

In [49]:
start_time = time.time()
# user_jobName_node
df_user_job_node = merged_jobs_to_nodes.groupby(['user', 'job_name', 'node_id']).agg({'job_id_string': 'size', 'core_avg':'mean'}).rename(columns={'job_id_string':'count','core_avg':'mean_core'}).reset_index()
print("--- %s seconds ---" % (time.time() - start_time))
print(df_user_job_node.shape[0])

# Drop a row by condition
df_user_job_node = df_user_job_node[df_user_job_node['mean_core'] != 0.0]
df_user_job_node.head(100)

--- 0.1727144718170166 seconds ---
31419


,user,job_name,node_id,mean_core,count
0,a07smr01,A2REPO,7,12.412156,1
1,a07smr01,A2REPO,33,6.442574,3
2,a07smr01,A2REPO,35,5.549144,68
3,a07smr01,A2REPO,46,2.749792,21
4,a07smr01,ARCHIVE,7,11.895912,1
5,a07smr01,ARCHIVE,33,9.347208,1
6,a07smr01,ARKI,33,5.766090,7
7,a07smr01,ARKI,35,4.523225,77
8,a07smr01,ARKI,46,4.144620,27
9,a07smr01,ARKI,54,4.120459,1


In [50]:
jobs_train.shape

(286478, 29)

In [51]:
# user_jobName
start_time = time.time()
df_user_job = merged_jobs_to_nodes.groupby(['user', 'job_name']).agg({'job_id_string': 'size', 'core_avg':'mean'}).rename(columns={'job_id_string':'count','core_avg':'mean_core'}).reset_index()
print("--- %s seconds ---" % (time.time() - start_time))
print(df_user_job.shape[0])
# Drop a row by condition
df_user_job = df_user_job[df_user_job['mean_core'] != 0.0]
df_user_job.head(5)

--- 0.08022332191467285 seconds ---
9122


,user,job_name,mean_core,count
0,a07smr01,A2REPO,5.019648,93
1,a07smr01,ARCHIVE,10.621560,2
2,a07smr01,ARKI,4.506037,112
3,a07smr01,INIT,6.031019,105
4,a07smr01,LM28_OPE,20.957776,1318


In [52]:
# user_node
start_time = time.time()
df_user_node = merged_jobs_to_nodes.groupby(['user', 'node_id']).agg({'job_id_string': 'size', 'core_avg':'mean'}).rename(columns={'job_id_string':'count','core_avg':'mean_core'}).reset_index()
print("--- %s seconds ---" % (time.time() - start_time))
print(df_user_node.shape[0])
# Drop a row by condition
df_user_node = df_user_node[df_user_node['mean_core'] != 0.0]
df_user_node.head(5)

--- 0.0429530143737793 seconds ---
5267


,user,node_id,mean_core,count
0,a07smr01,1,13.377256,6
1,a07smr01,2,13.390850,6
2,a07smr01,6,18.405537,4
3,a07smr01,7,28.234849,3
4,a07smr01,33,7.261337,53


In [53]:
# user
start_time = time.time()
df_user = merged_jobs_to_nodes.groupby(['user']).agg({'job_id_string': 'size', 'core_avg':'mean'}).rename(columns={'job_id_string':'count','core_avg':'mean_core'}).reset_index()
print("--- %s seconds ---" % (time.time() - start_time))
print(df_user.shape[0])
# Drop a row by condition
df_user = df_user[df_user['mean_core'] != 0.0]
df_user.head(5)

--- 0.03675270080566406 seconds ---
272


,user,mean_core,count
0,a07smr01,14.109222,7554
1,a08tra01,28.562404,14
2,a08tra11,137.803705,4
3,a08tra30,16.518865,2
4,a08tra35,30.838205,12


In [54]:
# node
start_time = time.time()
df_node = merged_jobs_to_nodes.groupby(['node_id']).agg({'job_id_string': 'size', 'core_avg':'mean'}).rename(columns={'job_id_string':'count','core_avg':'mean_core'}).reset_index()
print("--- %s seconds ---" % (time.time() - start_time))
print(df_node.shape[0])
# Drop a row by condition
df_node = df_node[df_node['mean_core'] != 0.0]
df_node.head(5)

--- 0.0151824951171875 seconds ---
63


,node_id,mean_core,count
0,1,9.583728,1519
1,2,8.073843,239
2,3,15.963822,1084
3,4,16.201217,1246
4,5,19.562831,481


# Convert to python dictionary

In [55]:
import pandas as pd
import errno    
import os

infile_user_job_node = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_job_node_whole"
infile_user_job_node = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_job_node_whole_clear"
user_job_node = pd.read_csv(infile_user_job_node + ".csv", index_col=0)
user_job_node.head(5)

,user,job_name,node_id,mean_core,count
0,a07smr01,A2REPO,7,12.412156,1
1,a07smr01,A2REPO,33,6.442574,3
2,a07smr01,A2REPO,35,5.549144,68
3,a07smr01,A2REPO,46,2.749792,21
4,a07smr01,ARCHIVE,7,11.895912,1


some info

In [56]:
user_job_node.mean_core.max()

396.73904226535007

We want to obtain 5 dataframes:
1. node
2. user
3. user_node
4. user_job
5. user_job_node

In [57]:
# creo il dizionario applicando raggruppamenti
my_dict = user_job_node.groupby(['user'])[['job_name', 'node_id', 'mean_core', 'count']].apply(lambda g: g.groupby(['job_name'])[['node_id', 'mean_core', 'count']].apply(lambda g1: g1.groupby(['node_id'])[['mean_core', 'count']].apply(lambda g2: g2.values.tolist()[0]).to_dict()).to_dict()).to_dict()
#my_dict

In [58]:
# aggiungo al dizionario i metodi che prevedono valori medi

user_dict = my_dict

for user_key, user_values in user_dict.items():
    user_job_dict = user_dict[user_key]
    for user_job_key, user_job_values in user_job_dict.items():
        if(user_key != 'node_avg' and user_job_key != 'user_node_avg' and user_job_key != 'user_avg'): # [user-job]
            
            mean_core_user_job = df_user_job_node.loc[(df_user_job_node['user']==user_key) & (df_user_job_node['job_name'] == user_job_key)]
            if(mean_core_user_job.shape[0] != 0):  # if != 0 than user and job name exist on hystoric data
                user_job_mean_core_wholeNodes = np.average(mean_core_user_job['mean_core'], weights=mean_core_user_job['count'])
                user_job_counts = mean_core_user_job['count'].sum()
                user_dict[user_key][user_job_key]['user_job_avg'] = list([user_job_mean_core_wholeNodes, user_job_counts])
    
    mean_core_user = df_user_job_node.loc[(df_user_job_node['user']==user_key)]
    if(mean_core_user.shape[0] != 0):  # if != 0 than user and job name exist on hystoric data
        user_mean_core_wholeNodes = np.average(mean_core_user['mean_core'], weights=mean_core_user['count'])
        user_counts = mean_core_user['count'].sum()
        user_dict[user_key]['user_avg'] = list([user_mean_core_wholeNodes, user_counts])
        
        
    user_dict[user_key]['user_node_avg'] = {}
    for i in range(1, 65):
        mean_core_user_node = df_user_job_node.loc[(df_user_job_node['user']==user_key) & (df_user_job_node['node_id'] == i)]
        if(mean_core_user_node.shape[0] != 0):  # if != 0 than user and job name exist on hystoric data
            user_node_mean_core_wholeNodes = np.average(mean_core_user_node['mean_core'], weights=mean_core_user_node['count'])
            user_node_counts = mean_core_user_node['count'].sum()
            user_dict[user_key]['user_node_avg'][i] = list([user_node_mean_core_wholeNodes, user_node_counts])
    
user_dict['node_avg'] = {}
for i in range(1, 65):
    mean_core_node = df_user_job_node.loc[(df_user_job_node['node_id'] == i)]
    if(mean_core_node.shape[0] != 0):  # if != 0 than user and job name exist on hystoric data
        node_mean_core_wholeNodes = np.average(mean_core_node['mean_core'], weights=mean_core_node['count'])
        node_counts = mean_core_node['count'].sum()
        user_dict['node_avg'][i] = list([node_mean_core_wholeNodes, node_counts])


In [59]:
user_dict

{'a07smr01': {'A2REPO': {33: [6.442574162552602, 3.0],
   35: [5.549143712494898, 68.0],
   46: [2.7497916869636483, 21.0],
   7: [12.412155820348273, 1.0],
   'user_job_avg': [5.019648131009632, 93]},
  'ARCHIVE': {33: [9.347207900221768, 1.0],
   7: [11.895912125535574, 1.0],
   'user_job_avg': [10.621560012878671, 2]},
  'ARKI': {33: [5.76609015147425, 7.0],
   35: [4.523224958568197, 77.0],
   46: [4.144619781996163, 27.0],
   54: [4.120458533687631, 1.0],
   'user_job_avg': [4.506037013550491, 112]},
  'INIT': {33: [6.0283451828177075, 8.0],
   35: [5.67017597377747, 71.0],
   37: [23.0829566960988, 1.0],
   46: [4.096130377156768, 20.0],
   48: [4.521760964721512, 4.0],
   51: [59.355148528859424, 1.0],
   'user_job_avg': [6.03101916407354, 105]},
  'LM28_OPE': {1: [23.289646121196803, 1.0],
   2: [23.289646121196803, 1.0],
   33: [19.07357273258673, 2.0],
   34: [15.995607813437607, 8.0],
   35: [14.931087910731486, 9.0],
   36: [14.734296912530198, 19.0],
   37: [22.77222483673

In [60]:
user_dict['a07smr01']['ARKI']

{33: [5.76609015147425, 7.0],
 35: [4.523224958568197, 77.0],
 46: [4.144619781996163, 27.0],
 54: [4.120458533687631, 1.0],
 'user_job_avg': [4.506037013550491, 112]}

In [61]:
type(my_dict)

dict

In [62]:
#dict['a07smr01']['ARCHIVE'][7][0][1] = 2

In [63]:
import pickle

outfile_user_job_node_dict = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_job_node_whole"
outfile_user_job_node_dict = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_job_node_whole_clear"

with open(outfile_user_job_node_dict + '.pkl', 'wb') as handle:
    pickle.dump(user_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(outfile_user_job_node_dict + '.pkl', 'rb') as handle:
    b = pickle.load(handle)

print(user_dict == b)

True


# convert to c++

In [8]:
merged_jobs_to_nodes.head()

,job_id_string,job_name,queue,start_time,run_start_time,end_time,user,node_req,cpu_req,mem_req,...,ncores_system,njobs_system,load_system,pred_perNode,pred_perUser,node_id,ncpus,ngpus,nmics,mem_requested
0,498459.node129,CPMD,parallel,2014-03-31 00:36:51,2014-03-31 00:36:51,2014-03-31 00:40:13,framondo,1,16,14,...,32,2,0,242.506310,375.626147,9,16,0,0,14680064
1,498460.node129,7ribb-relax,parallel,2014-03-31 00:54:37,2014-03-31 00:54:38,2014-03-31 04:55:17,vcalzia0,6,96,84,...,112,7,0,1503.563278,1079.566183,9,16,0,0,14680064
2,498460.node129,7ribb-relax,parallel,2014-03-31 00:54:37,2014-03-31 00:54:38,2014-03-31 04:55:17,vcalzia0,6,96,84,...,112,7,0,1503.563278,1079.566183,28,16,0,0,14680064
3,498460.node129,7ribb-relax,parallel,2014-03-31 00:54:37,2014-03-31 00:54:38,2014-03-31 04:55:17,vcalzia0,6,96,84,...,112,7,0,1503.563278,1079.566183,25,16,0,0,14680064
4,498460.node129,7ribb-relax,parallel,2014-03-31 00:54:37,2014-03-31 00:54:38,2014-03-31 04:55:17,vcalzia0,6,96,84,...,112,7,0,1503.563278,1079.566183,19,16,0,0,14680064


In [13]:
data = merged_jobs_to_nodes[['user', 'job_name', 'cpu_req', 'node_id', 'ncpus', 'real_pow']]

In [14]:
data

,user,job_name,cpu_req,node_id,ncpus,real_pow
0,framondo,CPMD,16,9,16,213.907235
1,vcalzia0,7ribb-relax,96,9,16,1232.722332
2,vcalzia0,7ribb-relax,96,28,16,1232.722332
3,vcalzia0,7ribb-relax,96,25,16,1232.722332
4,vcalzia0,7ribb-relax,96,19,16,1232.722332
5,vcalzia0,7ribb-relax,96,26,16,1232.722332
6,vcalzia0,7ribb-relax,96,29,16,1232.722332
7,alupi000,rth2e6t0DC,16,27,16,204.154343
8,a07smr01,INIT,1,35,1,4.619396
9,a07smr01,LM_PRE,176,36,16,3437.597435


In [15]:
outfile_convert_dict = datadir + "CPUs/" + interval_comment + "/convert_dict.csv"
data.to_csv(outfile_convert_dict)